In [1]:
!pip install transformers torch nltk rouge_score

In [1]:
import math
import random
import sys
import os
# from framework.utilities.models import inference    
from framework.utilities.metrics import calculate_bleu, calculate_rouge
import fastText
from contextlib import contextmanager
import time
import json
import wandb
import torch.nn.functional as F
import torch

In [2]:
from power_monitoring.monitor import HWMonitor
import threading
import wandb

In [3]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
# combined_outputs["gsm8k"] = [{"input_text": None, "7b": None, "tiny": None, "13b": None} for _ in range(3000)]
# len(combined_outputs["gsm8k"])

# with open('data/input_output_train', 'wb') as f:
#     pickle.dump(combined_outputs, f)

In [5]:
import pickle
import pandas as pd

with open('data/input_output_train', 'rb') as file:
    combined_outputs = pickle.load(file)

df_combined_outputs = pd.DataFrame(combined_outputs)
df_combined_outputs.head()

,wmt14,cnn_dailymail,gsm8k
0,{'input_text': 'Gutach: Noch mehr Sicherheit f...,{'input_text': '(CNN)The Palestinian Authority...,"{'input_text': None, '7b': None, 'tiny': None,..."
1,{'input_text': 'Sie stehen keine 100 Meter von...,{'input_text': '(CNN)Never mind cats having ni...,"{'input_text': None, '7b': None, 'tiny': None,..."
2,{'input_text': 'Zwei Anlagen so nah beieinande...,{'input_text': '(CNN)If you've been following ...,"{'input_text': None, '7b': None, 'tiny': None,..."
3,{'input_text': 'Diese Frage hat Gutachs Bürger...,{'input_text': '(CNN)Five Americans who were m...,"{'input_text': None, '7b': None, 'tiny': None,..."
4,"{'input_text': '""Die Rathausampel ist damals i...",{'input_text': '(CNN)A Duke student has admitt...,"{'input_text': None, '7b': None, 'tiny': None,..."


In [6]:
df_combined_outputs.size

9000

In [7]:
df_combined_outputs["cnn_dailymail"][3]

{'input_text': "(CNN)Five Americans who were monitored for three weeks at an Omaha, Nebraska, hospital after being exposed to Ebola in West Africa have been released, a Nebraska Medicine spokesman said in an email Wednesday. One of the five had a heart-related issue on Saturday and has been discharged but hasn't left the area, Taylor Wilson wrote. The others have already gone home. They were exposed to Ebola in Sierra Leone in March, but none developed the deadly virus. They are clinicians for Partners in Health, a Boston-based aid group. They all had contact with a colleague who was diagnosed with the disease and is being treated at the National Institutes of Health in Bethesda, Maryland. As of Monday, that health care worker is in fair condition. The Centers for Disease Control and Prevention in Atlanta has said the last of 17 patients who were being monitored are expected to be released by Thursday. More than 10,000 people have died in a West African epidemic of Ebola that dates to 

In [8]:
def algorithm(T, c):
    l_predictor, s_predictor = None, None
    results = []
    
    for t in range(1, T+1):
        print(f"Index: {t}")
        t_i = df_combined_outputs["wmt14"][t]["input_text"]
        t_e = df_combined_outputs["wmt14"][t]["label"]

        p_t = min(1, c/math.sqrt(t))
        X_t = Bernoulli(p_t)

        l_loss, s_loss = None, None
        if X_t == 1:
            t_c = getResults(t, t_i, t_e)
            l_predictor, s_predictor, l_loss, s_loss = sgdStep(t_c)
            l_predictor, s_predictor = checkpoint(l_predictor, s_predictor, t, p_t)
            
        output, t_acc = queryBest(t, t_i, t_e, l_predictor, s_predictor)
        log(results, t, t_i, output, t_acc, l_loss, s_loss)

    return results

In [9]:
def log(results, t, t_i, output, t_acc, l_loss, s_loss):
    results.append({
        'iteration': t,
        'input': t_i,
        'best_output': output,
        'chosen_model_accuracy': t_acc,
        'l_loss': l_loss,
        's_loss': s_loss
    })
    log_data = {
        'iteration': t,
        'chosen_model_accuracy': t_acc
    }
        
    if l_loss is not None: log_data['l_loss'] = l_loss
    if s_loss is not None: log_data['s_loss'] = s_loss
        
    wandb.log(log_data)

In [10]:
def Bernoulli(p_t):
    return random.random() < p_t 

In [11]:
def queryBest(t, t_i, t_e, l_predictor, s_predictor):
    l_acc, s_acc = predict(t_i, l_predictor, s_predictor)
    if l_acc > s_acc: # use larger model, e.g. 7b llama
        output = df_combined_outputs["wmt14"][t]["13b"]
    else: # use smaller model for all other cases, e.g. 3b llama
        output =  df_combined_outputs["wmt14"][t]["tiny"]

    return output, calculate_bleu(output, t_e)
    # return output, calculate_rouge(output, t_e)

In [12]:
def compute_mse_loss(predicted_accuracy, true_accuracy):
    predicted_tensor = torch.tensor([predicted_accuracy], dtype=torch.float32)
    true_tensor = torch.tensor([true_accuracy], dtype=torch.float32)

    loss = F.mse_loss(predicted_tensor, true_tensor)
    
    return loss.item()

In [13]:
def sgdStep(t_c):
    with open("fasttext_large.txt", "w") as f:
        f.write(f"__label__{t_c['large_model_accuracy']} {t_c['input_text']}\n")
    with open("fasttext_small.txt", "w") as f:
        f.write(f"__label__{t_c['small_model_accuracy']} {t_c['input_text']}\n")

    if os.path.exists("large_predictor.bin"):
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="large_predictor.bin")
    else:
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2)

    if os.path.exists("small_predictor.bin"):
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="small_predictor.bin")
    else:
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2)

    predicted_large_label = large_model_predictor.predict(t_c['input_text'])[0][0].replace("__label__", "")
    predicted_small_label = small_model_predictor.predict(t_c['input_text'])[0][0].replace("__label__", "")

    large_model_loss = compute_mse_loss(float(predicted_large_label), t_c['large_model_accuracy'])
    small_model_loss = compute_mse_loss(float(predicted_small_label), t_c['small_model_accuracy'])

    return large_model_predictor, small_model_predictor, large_model_loss, small_model_loss

In [14]:
def predict(text, l_predictor, s_predictor):
    l_predicted_label = l_predictor.predict(text)[0][0]
    l_predicted_accuracy = int(round(float(l_predicted_label.replace('__label__', ''))))

    s_predicted_label = s_predictor.predict(text)[0][0]
    s_predicted_accuracy = int(round(float(s_predicted_label.replace('__label__', ''))))

    return l_predicted_accuracy, s_predicted_accuracy

In [15]:
def getResults(t, t_i, t_e):
    s_output = df_combined_outputs["wmt14"][t]["tiny"]
    l_output = df_combined_outputs["wmt14"][t]["13b"]
    
    # WMT14
    s_acc = calculate_bleu(s_output, t_e)
    l_acc = calculate_bleu(l_output, t_e)

    # CNN Dailymail
    # s_acc = calculate_rouge(s_output, t_e)['rouge1']
    # l_acc = calculate_rouge(l_output, t_e)['rouge1']

    return {
        'input_text': t_i,
        'large_model_accuracy': l_acc,
        'small_model_accuracy': s_acc
    }

In [16]:
def checkpoint(l_predictor, s_predictor, t, p_t):
    if p_t > 0.5:
        if t % int(math.sqrt(t)) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)
    else:
        if t % max(1, int(1 / math.sqrt(t))) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)

    return l_predictor, s_predictor

In [17]:
def save_models(l_predictor, s_predictor):
    if l_predictor:
        l_predictor.save_model("large_predictor.bin")
    if s_predictor:
        s_predictor.save_model("small_predictor.bin")

    return l_predictor, s_predictor

In [18]:
def run_and_record(samples, c):
    wandb.init(project="classifier")
    stop_event = threading.Event()
    hw_monitor = HWMonitor(monitoring_freq=1.0, stop_event=threading.Event())
    hw_monitor.start()
    
    results = algorithm(samples, c)

    stop_event.set() 
    hw_monitor.join()
    wandb.finish()

    return results

In [ ]:
results = run_and_record(2999, 5)

with open('experiments/fasttext/wmt14_results_5.json', 'w') as f:
    json.dump(results, f, indent=4)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ryanzhangofficial (ryzhangofficial). Use `wandb login --relogin` to force relogin


Index: 1


/bin/sh: 1: nvidia-smi: not found
Read 0M words
Number of words:  27
Number of labels: 1
/bin/sh: 1: nvidia-smi: not found
/bin/sh: 1: nvidia-smi: not found
start training...
Progress: 100.0% words/sec/thread:   36436 lr:  0.000000 loss:  0.000000 ETA:   0h 0m0m
Read 0M words
Number of words:  27
Number of labels: 1
/bin/sh: 1: nvidia-smi: not found
/bin/sh: 1: nvidia-smi: not found
/bin/sh: 1: nvidia-smi: not found
start training...
Progress: 100.0% words/sec/thread:   44142 lr:  0.000000 loss:  0.000000 ETA:   0h 0m0m


Checkpoint
Index: 2


/bin/sh: 1: nvidia-smi: not found
Read 0M words
Number of words:  9
Number of labels: 1
/bin/sh: 1: nvidia-smi: not found
/bin/sh: 1: nvidia-smi: not found
start training...
Progress: 100.0% words/sec/thread:   17762 lr:  0.000000 loss:  0.000000 ETA:   0h 0m0m
Read 0M words
Number of words:  9
Number of labels: 1
/bin/sh: 1: nvidia-smi: not found
/bin/sh: 1: nvidia-smi: not found

Checkpoint
Index: 3
Checkpoint
Index: 4
Checkpoint
Index: 5
Index: 6
Checkpoint
Index: 7
Index: 8
Checkpoint
Index: 9
Checkpoint
Index: 10
Index: 11
Index: 12
Checkpoint
Index: 13
Index: 14
Index: 15
Checkpoint
Index: 16
Checkpoint
Index: 31
Index: 32
Index: 33
Index: 34
Index: 35
Checkpoint
Index: 36
Index: 37
Index: 38
Index: 39
Index: 40
Index: 41
Index: 42
Checkpoint
Index: 43
Index: 44
Index: 45
Index: 46
Index: 47
Index: 48
Checkpoint
Index: 49
Checkpoint
Index: 50
Index: 51
Index: 52
Index: 53
Index: 54
Index: 55
Index: 56
Checkpoint
Index: 57
Index: 58
Index: 59
Index: 60
Index: 61
Index: 62
Index: 63
Checkpoint
Index: 64
Index: 65
Index: 66
Index: 67
Index: 68
Index: 69
Index: 70
Index: 71
Index: 72
Index: 73
Index: 74
Index: 75
Index: 76
Index: 77
Index: 78
Index: 79
Index: 80
Checkpoint
Index: 81
Checkpoint
Index: 82
Index: 83
Index: 84
Index: 85
Index: 86
Index: 87
Index: 88
Index: 89
Index: 90
Checkpoint
Index: 91
Index: 92
Index: 93
Index: 94
Index: 95
Index: 96
Index: 97

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [ ]:
results = run_and_record(2999, 3)

with open('experiments/fasttext/wmt14_results_3.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
results = run_and_record(2999, 1)

with open('experiments/fasttext/wmt14_results_1.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
import json

with open('data/wmt14_results.json', 'r') as f:
    data = json.load(f)

for result in data[:10]:
    print(result['chosen_model_accuracy'])

In [5]:
total = 0
num_results = len(data)

for result in data:
    total += result['chosen_model_accuracy']

total / num_results

0.18766023212954513

In [16]:
with open('data/cnndailymail_results.json', 'r') as f:
    data = json.load(f)

In [17]:
rouge1_total = 0
rouge2_total = 0
rougeL_total = 0

num_results = len(data)

for result in data:
    rouge1_total += result['chosen_model_accuracy']['rouge1']
    rouge2_total += result['chosen_model_accuracy']['rouge2']
    rougeL_total += result['chosen_model_accuracy']['rougeL']

rouge1_avg = rouge1_total / num_results
rouge2_avg = rouge2_total / num_results
rougeL_avg = rougeL_total / num_results

print(f"Average ROUGE-1: {rouge1_avg}")
print(f"Average ROUGE-2: {rouge2_avg}")
print(f"Average ROUGE-L: {rougeL_avg}")

Average ROUGE-1: 0.2937870737089562
Average ROUGE-2: 0.10789698165405123
Average ROUGE-L: 0.19841589354653624
